<a href="https://colab.research.google.com/github/sota1111/DL_Zero4/blob/main/DL_Zero4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
os.chdir('/content/drive/MyDrive/DL_Zero')

Mounted at /content/drive


In [3]:
!git config --global user.email "sota.moro@gmail.com"
!git config --global user.name "sota1111"

In [1]:
!pip install numpy matplotlib ipython scikit-learn pandas pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

# 1章 バンディット問題

## 1.1 機械学習の分類と強化学習

### 1.1.1 教師あり学習

### 1.1.2 教師なし学習

### 1.1.3 強化学習

## 1.2 バンディット問題

### 1.2.1 バンディット問題とは
バンディット：スロットマシンの別称  
バンディット問題における強化学習の枠組みを考える  
- 環境：スロットマシン
- エージェント：プレイヤー
- 行動：プレイヤーは複数台あるスロットマシンから1台を選んでプレイする。  
- 報酬：行動の結果、スロットマシンから受け取るコイン

### 1.2.2 良いスロットマシンとは
スロットマシンをプレイするような確率的事象は「期待値」を使って評価できる。

### 1.2.3 数式を使って表す
確率変数：取る値が確率的に決まる変数
- A：エージェントの行う行動
- E：期待値
- R：報酬
- E[R]：報酬Rの期待値
- E[R|A]：Aという行動を選んだ場合の報酬の期待値  
| の右側に条件を記述
- Q:行動価値
q(A)=E[R|A]

## 1.3 バンディットアルゴリズム
プレイヤーは報酬の期待値が分からない状態で、できるだけ精度良く推定することが求められる。

## 1.4 価値の推定方法
スロットマシンを実際にプレイして得られた報酬→標本平均

### 1.3.2 平均値を求める実装

In [9]:
import numpy as np

np.random.seed(0)
rewards = []

for n in range(1, 11):# 1~10まで
  reward = np.random.rand() # ダミーの報酬
  rewards.append(reward)
  Q = sum(rewards) / n
  print(Q)

0.5488135039273248
0.6320014351498722
0.6222554154571295
0.6029123573420713
0.567060845741438
0.5801997236289743
0.5598265075766483
0.6013198192273272
0.6415801460355164
0.6157662833145425


Qn-1 = (R1+R2+...+Rn-1)/(n-1)を用いて  
Qn   = (R1+R2+...+Rn)nを変形すると、  
Qn = Qn-1+1/n*(Rn-Qn-1)
行動価値Qnは前回の行動価値と今回の報酬によって決まる。  
ここで、1/nは学習率としての役割がある。  


In [11]:
Q = 0

for n in range(1,11):
  reward = np.random.rand()
  Q = Q + (reward - Q)/n
  print(Q)

0.7917250380826646
0.6603099789177845
0.6295548396431672
0.7035652893055406
0.5770594430840099
0.49540441918693173
0.4275207017945595
0.4781580947637318
0.5114912787844116
0.5473433657306523


### 1.3.3 プレイヤーの戦略
- 活用：プレイした結果を利用して、最善と思われるスロットマシンをプレイ（=greedyな行動)
- 探索：スロットマシンの価値を精度良く推定するために、様々なスロットマシンを試す
活用と探索はトレードオフ  
ε-greeedy法：最も基本的で応用の聞くアルゴリズム。  
εの確率で活用を行う。

## 1.4 バンディットアルゴリズムの実装

### 1.4.1 スロットマシンの実装

In [16]:
import numpy as np

class Bandit:
  def __init__(self, arms=10):
    self.rates = np.random.rand(arms) # 各マシンの勝率

  def play(self, arm):
    rate = self.rates[arm]
    if rate > np.random.rand():
      return 1
    else:
      return 0

In [20]:
bandit = Bandit()

for i in range(3):
  print(bandit.play(0))

0
1
0


### 1.4.2 エージェントの実装

In [24]:
bandit = Bandit()
Q = 0

for n in range(1,11):
  reward = bandit.play(0) #0番目のマシンをプレイ
  Q += (reward - Q) / n
  print(Q)

0.0
0.0
0.3333333333333333
0.25
0.4
0.33333333333333337
0.28571428571428575
0.25000000000000006
0.22222222222222227
0.20000000000000004


In [39]:
# 上のプログラムを配列化しただけ
bandit = Bandit()
Qs = np.zeros(10)
ns = np.zeros(10)

for n in range(10):
  action = np.random.randint(1, 10) #ランダムな行動
  reward = bandit.play(action)

  ns[action] += 1
  Qs[action] += (reward - Qs[action]) / ns[action]
  print(Qs) 

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
[0.  1.  0.  0.  0.  0.  0.  0.  0.5 1. ]
[0.  1.  0.  0.  0.  0.  0.  0.  0.5 1. ]


In [40]:
class Agent:
  def __init__(self, epsilon, action_size=10):
    self.epsilon = epsilon
    self.Qs = np.zeros(action_size)
    self.ns = np.zeros(action_size)

  def update(self, action, reward):
    self.ns[action] += 1
    self.Qs[action] += (reward - self.Qs[action]) / self.ns[action]

  def get_action(self):
    if np.random.rand() < self.epsilon:
      return np.random.randint(0, len(self.Qs))#epsilon 
    return np.argmax(self.Qs)#greedy